In [ ]:
import random
import numpy as np

import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
from fiftyone import ViewField as F

In [ ]:
ds = fo.load_dataset('ds_al_chat')

In [ ]:
ds = foz.load_zoo_dataset('quickstart')

In [ ]:
fob.compute_similarity(
    ds,
    model="clip-vit-base32-torch",
    brain_key="img_sim",
)

In [ ]:
# Index ground truth objects by similarity
fob.compute_similarity(
    ds,
    patches_field="ground_truth",
    model="resnet18-imagenet-torch",
    brain_key="gt_sim",
)

In [ ]:
fob.compute_uniqueness(ds)


In [ ]:
ds.name = 'ds_al_chat'
ds.persistent=True

In [ ]:
res = ds.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval_det",
)

In [ ]:
res = ds.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval_det_iou08",
    iou=0.8,
)

In [ ]:
res = ds.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval_det_oi",
    method='open-images',
)

In [ ]:
CLSES = [str(x) for x in range(5)]

def jitter(val):
    if random.random() < 0.10:
        return random.choice(CLSES)

    return val

def newval():
    return random.choice(CLSES)

def newval_logits():
    logits = [random.random() for c in CLSES]
    lbl = CLSES[np.argmax(logits)]
    conf = random.random()
    return lbl, conf, logits
    

In [ ]:
#vals_gt = [newval() for s in ds]
#vals_pd = [jitter(x) for x in vals_gt]

vals_gt = [newval_logits() for s in ds]
vals_pd = [newval_logits() for s in ds]
vals_pd2 = [newval_logits() for s in ds]

vals_gt = [fo.Classification(label=l,confidence=c,logits=log) for l,c,log in vals_gt]
vals_pd = [fo.Classification(label=l,confidence=c,logits=log) for l,c,log in vals_pd]
vals_pd2 = [fo.Classification(label=l,confidence=c,logits=log) for l,c,log in vals_pd2]

ds.set_values('cls_gt',vals_gt)
ds.set_values('cls_pd',vals_pd)
ds.set_values('cls_pd2',vals_pd2)

In [ ]:
ds.save()
print(ds)
s=ds.first()

In [ ]:
# clsification eval: 
results = ds.evaluate_classifications(
    "cls_pd",
    gt_field="cls_gt",
    eval_key="eval_cls",
)

results = ds.evaluate_classifications(
    "cls_pd2",
    gt_field="cls_gt",
    eval_key="eval_cls2",
)


In [ ]:
# topk
results = ds.evaluate_classifications(
    "cls_pd",
    gt_field="cls_gt",
    eval_key="eval_cls_topk",
    method="top-k",
    classes=CLSES,
    k=3)

results = ds.evaluate_classifications(
    "cls_pd2",
    gt_field="cls_gt",
    eval_key="eval_cls2_topk",
    method="top-k",
    classes=CLSES,
    k=3)

In [ ]:
fob.compute_hardness(ds,'cls_pd')

In [ ]:
ifos = [ds.get_evaluation_info(x) for x in ds.list_evaluations()]
print(ifos)

In [ ]:
list_evaluations
i=ds.get_evaluation_info()
#h=ds.has_evaluaion
load_evaluation_results
laod_evaluation_view
to_evaluation_pathes

### test 1: multiple pred fields


In [ ]:
ds = foz.load_zoo_dataset('quickstart')

In [ ]:
for s in ds:
    dets = s.predictions.detections.copy()
    s['new_preds'] = fo.Detections(detections=dets)
    s.save()
print(ds)

In [ ]:
res0 = ds.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval_preds",
)
res1 = ds.evaluate_detections(
    "new_preds",
    gt_field="ground_truth",
    eval_key="eval_new",
)

In [ ]:
print(ds.list_evaluations())
ds.name='ds_chat_2detpreds'
ds.persistent=True

### test 2: multiple gt fields


In [1]:

ds = foz.load_zoo_dataset('quickstart')
for s in ds:
    dets = s.ground_truth.detections.copy()
    s['gt_new'] = fo.Detections(detections=dets)
    s.save()
print(ds)
res0 = ds.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
)
res1 = ds.evaluate_detections(
    "predictions",
    gt_field="gt_new",
    eval_key="eval_newgt",
)
print(ds.list_evaluations())
ds.name='ds_chat_2detGTs'
ds.persistent=True

Dataset already downloaded
Loading 'quickstart'
 100% |█████████████████| 200/200 [2.1s elapsed, 0s remaining, 98.0 samples/s]          
Dataset 'quickstart' created
Name:        quickstart
Media type:  image
Num samples: 200
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    uniqueness:   fiftyone.core.fields.FloatField
    predictions:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    gt_new:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
Evaluating detections...
 100% |█████████████████| 200/200 [10.7s elapsed, 0s remaining, 13.6 samples/s]  

### test3: multiple IOU thresholds

In [2]:
ds = foz.load_zoo_dataset('quickstart')
res0 = ds.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval_det_normal",
    iou=0.6,
)
res1 = ds.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval_det_tight",
    iou=0.9,
)
print(ds.list_evaluations())
ds.name='ds_chat_2detIOUs'
ds.persistent=True

Dataset already downloaded
Loading 'quickstart'
 100% |█████████████████| 200/200 [2.0s elapsed, 0s remaining, 100.4 samples/s]         
Dataset 'quickstart' created
Evaluating detections...
 100% |█████████████████| 200/200 [10.6s elapsed, 0s remaining, 14.0 samples/s]      
Evaluating detections...
 100% |█████████████████| 200/200 [11.7s elapsed, 0s remaining, 12.7 samples/s]      
['eval_det_normal', 'eval_det_tight']


In [4]:
fo.list_datasets()

['ds_al_chat',
 'ds_chat_2detGTs',
 'ds_chat_2detIOUs',
 'ds_chat_2detpreds',
 'quickstart-video']